# Sandbox
For playing around during development.

In [1]:
%pylab inline
import pysd
print pysd.__version__
print pysd.__file__
import pandas as pd

Populating the interactive namespace from numpy and matplotlib
0.3.2
pysd/__init__.pyc


In [2]:
#mdl_file = 'tests/old_tests/vensim/Teacup.mdl'
#mdl_file = 'tests/test-models/tests/subscript_3d_arrays/test_subscript_3d_arrays.mdl'
#mdl_file = 'tests/test-models/tests/subscript_2d_arrays/test_subscript_2d_arrays.mdl'
#mdl_file = 'tests/test-models/tests/subscript_individually_defined_1_of_2d_arrays/subscript_individually_defined_1_of_2d_arrays.mdl'
mdl_file = 'tests/test-models/tests/subscript_individually_defined_1d_arrays/subscript_individually_defined_1d_arrays.mdl'
model = pysd.read_vensim(mdl_file)

In [3]:
%%time
model.run(flatten_subscripts=True)

CPU times: user 14.6 ms, sys: 4.45 ms, total: 19.1 ms
Wall time: 15.9 ms


,stock_a__entry_1,stock_a__entry_2,stock_a__entry_3
0,0.00,0.0,0.0
1,0.01,0.2,0.3
2,0.02,0.4,0.6
3,0.03,0.6,0.9
4,0.04,0.8,1.2
5,0.05,1.0,1.5
6,0.06,1.2,1.8
7,0.07,1.4,2.1
8,0.08,1.6,2.4
9,0.09,1.8,2.7


In [4]:
# for troubleshooting large models, just run everything once...
for key, func in model.components._dfuncs.iteritems():
    print func
    print func()


<function _dstock_a_dt at 0x10a1b2758>
[ 0.01  0.2   0.3 ]
